<a href="https://colab.research.google.com/github/BillJPG/Python_data_science_midterm/blob/main/Benchlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Project 15: Question Answering System for Legal Documents





Base LLM

Base LLM + RAG

Domain-Finetuned LLM

Domain-Finetuned LLM + RAG

In [16]:
!pip install transformers accelerate datasets sentencepiece einops rouge-score bert-score  -q



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.1 MB/s eta 0:00:00


In [10]:
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [11]:

BASE_MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"


EVAL_PATH = "eval_questions.jsonl"

MAX_NEW_TOKENS = 512
TEMPERATURE = 0.0

print("Model:", BASE_MODEL_NAME)
print("Eval file:", EVAL_PATH)


Model: Qwen/Qwen2.5-7B-Instruct
Eval file: eval_questions.jsonl


In [7]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)

print("Model loaded.")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model loaded.


In [12]:
eval_data = []

with open(EVAL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            eval_data.append(json.loads(line))

print("Loaded questions:", len(eval_data))
eval_data[:3]  # preview


Loaded questions: 20


[{'id': 1,
  'question': 'ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?',
  'gold_answer': 'ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.'},
 {'id': 2,
  'question': 'ถ้าบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร?',
  'gold_answer': 'การไม่จดทะเบียนพาณิชย์ถือเป็นการฝ่าฝืนหน้าที่ตามกฎหมาย อาจถูกปรับตามอัตราที่กำหนด รวมทั้งเสี่ยงถูกมองว่ามีเจตนาปกปิดข้อมูลธุรกิจต่อหน่วยงานรัฐ ซึ่งส่งผลต่อความน่าเชื่อถือในการประกอบธุรกิจ.'},
 {'id': 3,
  'question': 'ผู้ถือหุ้นในบริษัทมหาชนจำกัดสามารถร้องขอให้กรรมการเรียกประชุมวิสามัญได้เมื่อถือหุ้นจำนวนเท่าใด?',
  'gold_answer': 'โดยหลักต้องถือหุ้นรวมกันไม่น้อยกว่า 10% ของหุ้นที่จำหน่ายได้ทั้งหมด เพื่อ

In [13]:
def build_base_prompt(question: str) -> str:
    system = (
        "คุณเป็นผู้ช่วยด้านกฎหมายไทย "
        "ให้ตอบโดยอ้างอิงหลักกฎหมายไทยเท่าที่คุณมีความรู้ "
        "ถ้าไม่มั่นใจให้บอกว่าข้อมูลอาจไม่ครบถ้วน และหลีกเลี่ยงการเดาแบบไม่มีเหตุผล\n\n"
    )
    user_part = f"คำถาม:\n{question}\n\nโปรดตอบเป็นภาษาไทย:"
    return system + user_part


In [14]:
def run_eval_base(
    data,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
):
    outputs = []
    latencies = []

    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    for i, ex in enumerate(data, start=1):
        q = ex["question"]
        qid = ex.get("id", i)

        prompt = build_base_prompt(q)

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        if device == "cuda":
            torch.cuda.synchronize()

        start = time.time()
        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=(temperature > 0),
                temperature=temperature if temperature > 0 else 1.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        if device == "cuda":
            torch.cuda.synchronize()
        end = time.time()

        latency = end - start
        latencies.append(latency)

        generated = gen_ids[0][inputs["input_ids"].shape[1]:]
        answer_text = tokenizer.decode(generated, skip_special_tokens=True).strip()

        outputs.append({
            "id": qid,
            "question": q,
            "model_answer": answer_text,
            "gold_answer": ex.get("gold_answer", None),
            "latency_sec": latency,
        })

        if i % 10 == 0:
            print(f"Done {i}/{len(data)}")

    if device == "cuda":
        peak_mem_gb = torch.cuda.max_memory_allocated() / (1024 ** 3)
    else:
        peak_mem_gb = None

    summary = {
        "model_name": BASE_MODEL_NAME,
        "num_samples": len(data),
        "avg_latency_sec": sum(latencies) / len(latencies),
        "peak_vram_gb": peak_mem_gb,
    }

    return outputs, summary


In [15]:
base_outputs, base_summary = run_eval_base(eval_data)

print("==== BASE SUMMARY ====")
print(base_summary)

# Save results
OUTPUT_PATH = "results_base_model.jsonl"
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for ex in base_outputs:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("Saved to", OUTPUT_PATH)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Done 10/20
Done 20/20
==== BASE SUMMARY ====
{'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'num_samples': 20, 'avg_latency_sec': 30.82185903787613, 'peak_vram_gb': 14.235805034637451}
Saved to results_base_model.jsonl


In [17]:

from rouge_score import rouge_scorer
from bert_score import score as bertscore

preds = []
refs  = []

with open("results_base_model.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        ex = json.loads(line)
        if ex.get("gold_answer"):
            preds.append(ex["model_answer"])
            refs.append(ex["gold_answer"])

print("Samples with gold:", len(preds))

# ROUGE-L
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)
rougeL_scores = [scorer.score(r, p)["rougeL"].fmeasure for p, r in zip(preds, refs)]
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

# BERTScore (Thai)
P, R, F1 = bertscore(preds, refs, lang="th")
avg_bert_F1 = float(F1.mean())

print("ROUGE-L:", avg_rougeL)
print("BERTScore F1:", avg_bert_F1)


Samples with gold: 20


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

ROUGE-L: 0.00625
BERTScore F1: 0.7198852300643921


In [18]:
import json

with open("results_base_model.jsonl", "r", encoding="utf-8") as f:
    rows = [json.loads(l) for l in f]

for r in rows[:3]:
    print("Q:", r["question"])
    print("A (model):", r["model_answer"])
    print("A (gold) :", r["gold_answer"])
    print("latency:", r["latency_sec"])
    print("=" * 80)


Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
A (model): ตามประมวลกฎหมายแพ่งและพาณิชย์ ห้างหุ้นส่วนสามัญและห้างหุ้นส่วนจำกัดมีความแตกต่างในเรื่องความรับผิดของหุ้นส่วน ดังนี้:

1. ห้างหุ้นส่วนสามัญ (Sole Proprietorship):
- หุ้นส่วนจะมีความรับผิดชอบทางแพ่งทั้งหมดต่อหนี้สินของห้างหุ้นส่วน
- หมายความว่า หุ้นส่วนจะต้องใช้ทรัพย์สินส่วนตัวทั้งหมดในการชำระหนี้สินของห้างหุ้นส่วน
- หุ้นส่วนจะมีความรับผิดชอบอย่างไม่จำกัด

2. ห้างหุ้นส่วนจำกัด (Limited Partnership):
- ห้างหุ้นส่วนจำกัดประกอบด้วยหุ้นส่วนสามัญและหุ้นส่วนจำกัด
- หุ้นส่วนสามัญจะมีความรับผิดชอบทางแพ่งทั้งหมดต่อหนี้สินของห้างหุ้นส่วน
- หุ้นส่วนจำกัดจะมีความรับผิดชอบทางแพ่งตามทุนที่ลงทุนไว้

3. ห้างหุ้นส่วนจำกัด (Company Limited by Shares):
- หุ้นส่วนจะมีความรับผิดชอบทางแพ่งตามทุนที่ลงทุนไว้
- หมายความว่า หุ้นส่วนจะไม่ต้องใช้ทรัพย์สินส่วนตัวในการชำระหนี้สินของบริษัทหากบริษัทล้มละลาย

สรุปได้ว่า ห้างหุ้นส่วนสามัญมีความรับผิดชอบทางแพ่งทั้งหมดต่อหนี้สินของห้างหุ้นส่วน ขณะที่ห้าง

In [1]:
!pip install transformers accelerate datasets sentencepiece einops sentence-transformers faiss-cpu -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 126.8 MB/s eta 0:00:00


In [2]:
import json
import time
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [8]:

BASE_MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"


EVAL_PATH = "eval_questions.jsonl"

TOP_K = 3
BGE_MODEL_NAME = "BAAI/bge-m3"

#Generation parameters (i use same as base model)
MAX_NEW_TOKENS = 512
TEMPERATURE = 0.0

print("Generator model:", BASE_MODEL_NAME)
print("Retriever model:", BGE_MODEL_NAME)
print("Eval file:", EVAL_PATH)


Generator model: Qwen/Qwen2.5-7B-Instruct
Retriever model: BAAI/bge-m3
Eval file: eval_questions.jsonl


In [9]:
eval_data = []
with open(EVAL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            eval_data.append(json.loads(line))
print("Loaded eval samples:", len(eval_data))
eval_data[:3]


Loaded eval samples: 20


[{'id': 1,
  'question': 'ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?',
  'gold_answer': 'ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.'},
 {'id': 2,
  'question': 'ถ้าบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร?',
  'gold_answer': 'การไม่จดทะเบียนพาณิชย์ถือเป็นการฝ่าฝืนหน้าที่ตามกฎหมาย อาจถูกปรับตามอัตราที่กำหนด รวมทั้งเสี่ยงถูกมองว่ามีเจตนาปกปิดข้อมูลธุรกิจต่อหน่วยงานรัฐ ซึ่งส่งผลต่อความน่าเชื่อถือในการประกอบธุรกิจ.'},
 {'id': 3,
  'question': 'ผู้ถือหุ้นในบริษัทมหาชนจำกัดสามารถร้องขอให้กรรมการเรียกประชุมวิสามัญได้เมื่อถือหุ้นจำนวนเท่าใด?',
  'gold_answer': 'โดยหลักต้องถือหุ้นรวมกันไม่น้อยกว่า 10% ของหุ้นที่จำหน่ายได้ทั้งหมด เพื่อ

In [10]:
dataset = load_dataset("airesearch/WangchanX-Legal-ThaiCCL-RAG")
train_ds = dataset["train"]
print(train_ds)

corpus_texts = []
corpus_keys = []
seen_keys = set()

for ex in train_ds:
    for c in ex["positive_contexts"]:
        k = c.get("unique_key", None)
        if k is None:
            continue
        if k in seen_keys:
            continue
        seen_keys.add(k)
        corpus_keys.append(k)
        corpus_texts.append(c["context"])
    for c in ex["hard_negative_contexts"]:
        k = c.get("unique_key", None)
        if k is None:
            continue
        if k in seen_keys:
            continue
        seen_keys.add(k)
        corpus_keys.append(k)
        corpus_texts.append(c["context"])

len(corpus_texts), len(seen_keys)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8211 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3742 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'positive_contexts', 'hard_negative_contexts', 'positive_answer', 'hard_negative_answer'],
    num_rows: 8211
})


(4422, 4422)

In [11]:

bge = SentenceTransformer(BGE_MODEL_NAME, device=device)
bge.max_seq_length = 512

print("Encoding corpus with BGE...")

corpus_embeddings = bge.encode(
    [f"passage: {t}" for t in corpus_texts],
    batch_size=64,
    convert_to_numpy=True,
    show_progress_bar=True,
)


corpus_embeddings = corpus_embeddings.astype("float32")


faiss.normalize_L2(corpus_embeddings)

index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

print("Corpus size:", len(corpus_texts))
print("Index is trained and ready.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Encoding corpus with BGE...


Batches:   0%|          | 0/70 [00:00<?, ?it/s]

Corpus size: 4422
Index is trained and ready.


In [12]:
def retrieve_bge(query: str, top_k: int = TOP_K):

    q_emb = bge.encode(
        [f"query: {query}"],
        batch_size=1,
        convert_to_numpy=True,
        show_progress_bar=False,
    ).astype("float32")

    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)

    retrieved = []
    for score, idx in zip(D[0], I[0]):
        retrieved.append({
            "score": float(score),
            "key": corpus_keys[idx],
            "context": corpus_texts[idx],
        })
    return retrieved


test_q = eval_data[0]["question"]
ret = retrieve_bge(test_q, top_k=3)
test_q, ret[0]["score"], ret[0]["context"][:200]


('ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?',
 0.6786898970603943,
 'ประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1025\nอันว่าห้างหุ้นส่วนสามัญนั้น คือห้างหุ้นส่วนประเภทซึ่งผู้เป็นหุ้นส่วนหมดทุกคนต้องรับผิดร่วมกันเพื่อหนี้ทั้งปวงของหุ้นส่วนโดยไม่มีจำกัด')

In [13]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)

print("Generator model loaded.")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Generator model loaded.


In [14]:
def build_rag_prompt(question: str, retrieved_contexts, max_ctx: int = TOP_K) -> str:
    """
    สร้าง prompt สำหรับ RAG-only:
    - ใส่ context ที่ retrieve ได้ด้านบน
    - ใส่คำสั่งให้ยึดตาม context เป็นหลัก
    """
    system = (
        "คุณเป็นผู้ช่วยด้านกฎหมายไทยสำหรับประชาชนทั่วไป "
        "ให้ตอบคำถามโดยอ้างอิงจาก 'บริบท' ที่ให้ด้านล่างเท่านั้น "
        "ถ้าในบริบทไม่มีข้อมูลเพียงพอ ให้ตอบอย่างระมัดระวังและบอกว่าข้อมูลอาจไม่ครบถ้วน "
        "ควรอธิบายให้เข้าใจง่าย และถ้าอ้างมาตราให้ยึดจากข้อความในบริบทเท่านั้น\n\n"
    )

    ctx_parts = []
    for i, ctx in enumerate(retrieved_contexts[:max_ctx], start=1):
        ctx_parts.append(f"[เอกสารที่ {i}]\n{ctx['context']}")
    ctx_block = "\n\n".join(ctx_parts)

    user_part = (
        f"บริบทจากเอกสารกฎหมาย:\n{ctx_block}\n\n"
        f"คำถาม:\n{question}\n\n"
        "โปรดตอบเป็นภาษาไทย โดยอ้างอิงจากบริบทด้านบน:"
    )

    return system + user_part


In [16]:
def run_eval_rag(
    data,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    top_k: int = TOP_K,
):
    outputs = []
    latencies = []

    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    for i, ex in enumerate(data, start=1):
        q = ex["question"]
        qid = ex.get("id", i)


        retrieved = retrieve_bge(q, top_k=top_k)


        prompt = build_rag_prompt(q, retrieved_contexts=retrieved, max_ctx=top_k)

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        if device == "cuda":
            torch.cuda.synchronize()
        start = time.time()

        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=(temperature > 0),
                temperature=temperature if temperature > 0 else 1.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        if device == "cuda":
            torch.cuda.synchronize()
        end = time.time()

        latency = end - start
        latencies.append(latency)

        generated = gen_ids[0][inputs["input_ids"].shape[1]:]
        answer_text = tokenizer.decode(generated, skip_special_tokens=True).strip()

        outputs.append({
            "id": qid,
            "question": q,
            "model_answer": answer_text,
            "gold_answer": ex.get("gold_answer", None),
            "latency_sec": latency,

            "retrieved": retrieved,
        })

        if i % 5 == 0:
            print(f"Done {i}/{len(data)}")

    if device == "cuda":
        peak_mem_gb = torch.cuda.max_memory_allocated() / (1024 ** 3)
    else:
        peak_mem_gb = None

    summary = {
        "model_name": BASE_MODEL_NAME,
        "retriever": BGE_MODEL_NAME,
        "num_samples": len(data),
        "avg_latency_sec": sum(latencies) / len(latencies),
        "peak_vram_gb": peak_mem_gb,
        "top_k": top_k,
    }

    return outputs, summary


In [17]:
rag_outputs, rag_summary = run_eval_rag(eval_data)

print("==== RAG+BGE SUMMARY ====")
print(rag_summary)

OUTPUT_PATH_RAG = "results_rag_bge.jsonl"
with open(OUTPUT_PATH_RAG, "w", encoding="utf-8") as f:
    for ex in rag_outputs:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("Saved to", OUTPUT_PATH_RAG)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Done 5/20
Done 10/20
Done 15/20
Done 20/20
==== RAG+BGE SUMMARY ====
{'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'retriever': 'BAAI/bge-m3', 'num_samples': 20, 'avg_latency_sec': 31.437293910980223, 'peak_vram_gb': 16.69938898086548, 'top_k': 3}
Saved to results_rag_bge.jsonl


In [18]:
!pip install rouge-score bert-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00


In [19]:
from rouge_score import rouge_scorer
from bert_score import score as bertscore

RESULT_PATH = "results_rag_bge.jsonl"

preds = []
refs  = []

with open(RESULT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        ex = json.loads(line)
        if ex.get("gold_answer"):
            preds.append(ex["model_answer"])
            refs.append(ex["gold_answer"])

print("Samples with gold:", len(preds))

scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)
rougeL_scores = [scorer.score(r, p)["rougeL"].fmeasure for p, r in zip(preds, refs)]
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

P, R, F1 = bertscore(preds, refs, lang="th")
avg_bert_F1 = float(F1.mean())

print("RAG+BGE ROUGE-L:", avg_rougeL)
print("RAG+BGE BERTScore F1:", avg_bert_F1)


Samples with gold: 20


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

RAG+BGE ROUGE-L: 0.0
RAG+BGE BERTScore F1: 0.7097052335739136


In [20]:
with open("results_rag_bge.jsonl", "r", encoding="utf-8") as f:
    rows = [json.loads(l) for l in f]

for r in rows[:3]:
    print("Q:", r["question"])
    print("A (model):", r["model_answer"])
    print("A (gold) :", r["gold_answer"])
    print("latency:", r["latency_sec"])
    print("=" * 80)

Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
A (model): ห้างหุ้นส่วนสามัญและห้างหุ้นส่วนจำกัดมีความแตกต่างในเรื่องความรับผิดของหุ้นส่วน ตามประมวลกฎหมายแพ่งและพาณิชย์ ดังนี้:

1. **ห้างหุ้นส่วนสามัญ** (ตามมาตรา 1025):
   - ผู้เป็นหุ้นส่วนหมดทุกคนต้องรับผิดร่วมกันเพื่อหนี้ทั้งปวงของห้างหุ้นส่วนโดยไม่มีจำกัด หมายความว่า ถ้าห้างหุ้นส่วนมีหนี้ หุ้นส่วนทุกคนต้องรับผิดร่วมกันทั้งหมด ไม่ว่าจะเป็นหุ้นส่วนคนเดียวหรือหลาย ๆ คน ไม่มีการแบ่งแยกความรับผิดตามจำนวนเงินที่ลงหุ้น.

2. **ห้างหุ้นส่วนจำกัด** (ตามมาตรา 1077 และ 1079):
   - มีผู้เป็นหุ้นส่วนสองจำพวก คือ
     - หุ้นส่วนที่มีจำกัดความรับผิด (จำพวกที่ 1): หุ้นส่วนประเภทนี้ต้องรับผิดเพียงไม่เกินจำนวนเงินที่ตนรับจะลงหุ้นในห้างหุ้นส่วน.
     - หุ้นส่วนที่ต้องรับผิดร่วมกันโดยไม่มีจำกัด (จำพวกที่ 2): หุ้นส่วนประเภทนี้ต้องรับผิดร่วมกันในบรรดาหนี้ของห้างหุ้นส่วนโดยไม่มีจำกัดจำนวน.
   - หากห้างหุ้นส่วนจำกัดยังไม่ได้จดทะเบียน (ตามมาตรา 1079) หุ้นส่วนทั้งหมดจะถือว่าเป็นหุ้นส่วนสามัญ ซึ่งต้อ

In [1]:
!pip install -q  accelerate peft bitsandbytes datasets wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.8 MB/s eta 0:00:00


In [2]:
!pip install -U "transformers>=4.44.0"

import transformers
print(transformers.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
4.57.3


In [3]:
import wandb

wandb.login()


WANDB_PROJECT = "Ponprapa-legal-chat"
WANDB_RUN_NAME = "qwen2.5-7b-legal-Ponprapa-v2"


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v-tarnwuttipong (v-tarnwuttipong-tokyo-international-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


raw_ds = load_dataset("airesearch/WangchanX-Legal-ThaiCCL-RAG")
raw_train = raw_ds["train"]
raw_eval  = raw_ds["test"]

print(raw_train[0].keys())
print("train size:", len(raw_train), "eval size:", len(raw_eval))

def preprocess(example):
    q = example["question"]
    a = example["positive_answer"]

    text = f"คำถาม: {q}\n\nคำตอบ: {a}"
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding=False,
    )
    return tokenized

train_ds = raw_train.map(
    preprocess,
    remove_columns=raw_train.column_names,
)
eval_ds = raw_eval.map(
    preprocess,
    remove_columns=raw_eval.column_names,
)

print(train_ds[0].keys())
print("Tokenized train:", len(train_ds), "Tokenized eval:", len(eval_ds))


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Generating test split:   0%|          | 0/3742 [00:00<?, ? examples/s]

dict_keys(['question', 'positive_contexts', 'hard_negative_contexts', 'positive_answer', 'hard_negative_answer'])
train size: 8211 eval size: 3742


Map:   0%|          | 0/8211 [00:00<?, ? examples/s]

Map:   0%|          | 0/3742 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
Tokenized train: 8211 Tokenized eval: 3742


In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame(raw_train)
df.head()

,instruction,output
0,ธุรกิจสถาบันการเงินและธุรกิจทางการเงินต่างกันอ...,ธุรกิจสถาบันการเงิน จะหมายความรวมถึงการประกอบธ...
1,ธนาคารแห่งประเทศไทยสามารถเสนอให้มีการตราพระราช...,เสนอได้ หากกิจการนั้นมีผลกระทบต่อระบบเศรษฐกิจข...
2,การระดมเงินจากการจำหน่ายหลักทรัพย์ให้เป็นไปตาม...,ให้เป็นไปตามกฎหมายว่าด้วยหลักทรัพย์และตลาดหลัก...
3,ประกาศของธนาคารแห่งประเทศไทยมีผลบังคับได้เมื่อ...,เมื่อได้ประกาศในราชกิจจานุเบกษาแล้วให้ใช้บังคั...
4,รัฐมนตรีว่าการกระทรวงการคลัง มีอำนาจอย่างไร,รัฐมนตรีว่าการกระทรวงการคลังมีอำนาจออกประกาศให...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8211 entries, 0 to 8210
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  8211 non-null   object
 1   output       8211 non-null   object
dtypes: object(2)
memory usage: 128.4+ KB


In [6]:
!pip install transformers>=4.44.0 accelerate peft bitsandbytes datasets -q


In [5]:
import torch, gc
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model


for name in ["model"]:
    if name in globals():
        del globals()[name]
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)


model.gradient_checkpointing_enable()
model.enable_input_require_grads()

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

trainable params: 2,523,136 || all params: 7,618,139,648 || trainable%: 0.0331


In [7]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling, Trainer

OUTPUT_DIR = "./qwen2.5-7b-legal-Ponprapa-v2"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,

)

collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,   # tokenized train
    eval_dataset=eval_ds,     # tokenized eval (split test)
    data_collator=collator,
)


In [8]:
import wandb

WANDB_PROJECT = "thai-legal-qwen7b"
WANDB_RUN_NAME = "qwen2.5-7b-legal-Ponprapa-v2"

wandb.login()
wandb.init(project=WANDB_PROJECT, name=WANDB_RUN_NAME)
train_result = trainer.train()

wandb.log({"train_loss_final": train_result.training_loss})


metrics = trainer.evaluate(eval_dataset=eval_ds)

wandb.log(metrics)


model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

wandb.finish()

print("Saved FT model to", OUTPUT_DIR)
print("Eval metrics:", metrics)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
100,0.848600
200,0.781900
300,0.774800
400,0.744900
500,0.745800
600,0.720200
700,0.724000
800,0.721200
900,0.722200
1000,0.717500


epoch,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
eval_loss,▁
eval_runtime,▁
eval_samples_per_second,▁
eval_steps_per_second,▁
train/epoch,▁▂▃▃▄▅▆▆▇███
+5,...


Saved FT model to ./qwen2.5-7b-legal-Ponprapa-v2
Eval metrics: {'eval_loss': 0.9209577441215515, 'eval_runtime': 192.8704, 'eval_samples_per_second': 19.402, 'eval_steps_per_second': 2.426, 'epoch': 2.0}


In [9]:
model.save_pretrained("./qwen2.5-7b-legal-Ponprapa-v2")
tokenizer.save_pretrained("./qwen2.5-7b-legal-Ponprapa-v2")


('./qwen2.5-7b-legal-Ponprapa-v2/tokenizer_config.json',
 './qwen2.5-7b-legal-Ponprapa-v2/special_tokens_map.json',
 './qwen2.5-7b-legal-Ponprapa-v2/chat_template.jinja',
 './qwen2.5-7b-legal-Ponprapa-v2/vocab.json',
 './qwen2.5-7b-legal-Ponprapa-v2/merges.txt',
 './qwen2.5-7b-legal-Ponprapa-v2/added_tokens.json',
 './qwen2.5-7b-legal-Ponprapa-v2/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("qwen2.5-7b-legal-Ponprapa-v2")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
)

from peft import PeftModel
model = PeftModel.from_pretrained(model, "qwen2.5-7b-legal-Ponprapa-v2")


In [10]:
!zip -r qwen2.5-7b-legal-Ponprapa-v2.zip qwen2.5-7b-legal-Ponprapa-v2


  adding: qwen2.5-7b-legal-Ponprapa-v2/ (stored 0%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/adapter_model.safetensors (deflated 8%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/ (stored 0%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/adapter_model.safetensors (deflated 8%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/rng_state.pth (deflated 26%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/optimizer.pt (deflated 8%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/tokenizer.json (deflated 81%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/adapter_config.json (deflated 57%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/special_tokens_map.json (deflated 69%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/scaler.pt (deflated 64%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/added_tokens.json (deflated 67%)
  adding: qwen2.5-7b-legal-Ponprapa-v2/checkpoint-1028/chat_template.jinja (deflated 71%)
  adding: qwe

In [11]:
from google.colab import files
files.download("qwen2.5-7b-legal-Ponprapa-v2.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:

ADAPTER_PATH = "qwen2.5-7b-legal-Ponprapa-v2"


BASE_MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

EVAL_PATH = "eval_questions.jsonl"

OUTPUT_RESULTS_PATH = "results_ft_model.jsonl"


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)


model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

print("Loaded FT model from", ADAPTER_PATH)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded FT model from qwen2.5-7b-legal-Ponprapa-v2


In [17]:
import json

eval_data = []
with open(EVAL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            eval_data.append(json.loads(line))

print("Loaded questions:", len(eval_data))
print(eval_data[0])


Loaded questions: 20
{'id': 1, 'question': 'ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?', 'gold_answer': 'ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.'}


In [19]:
import time

MAX_NEW_TOKENS = 256
TEMPERATURE = 0.0

def generate_answer_ft(question: str):
    prompt = f"คำถาม: {question}\n\nคำตอบ:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        start = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
        )
        end = time.time()

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "คำตอบ:" in decoded:
        answer = decoded.split("คำตอบ:", 1)[-1].strip()
    else:
        answer = decoded.strip()

    latency = end - start
    return answer, latency


In [20]:
ft_results = []
total_latency = 0.0

for i, ex in enumerate(eval_data, start=1):
    q = ex["question"]
    gold = ex["gold_answer"]

    ans, lat = generate_answer_ft(q)
    total_latency += lat

    ft_results.append({
        "question": q,
        "gold_answer": gold,
        "model_answer": ans,
        "latency": lat,
    })

    print(f"[{i}/{len(eval_data)}] latency={lat:.2f}s")
    print("Q:", q)
    print("A (model):", ans[:200].replace("\n", " "))
    print("A (gold):", gold[:200].replace("\n", " "))
    print("="*80)

avg_latency = total_latency / len(eval_data)
print("Average latency:", avg_latency, "sec")

with open(OUTPUT_RESULTS_PATH, "w", encoding="utf-8") as f:
    for row in ft_results:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Saved FT eval results to", OUTPUT_RESULTS_PATH)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[1/20] latency=22.77s
Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
A (model): คำตอบ: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วน โดยห้างหุ้นส่วนสามัญมีความรับผิดชอบที่จะต้องชำระหนี้ของห้างหุ้นส่วนนั้นทั้งหมด ไม่ว่าจะเป็นหนี้ที่เกิดขึ้นก่อนหรือหลังการจดทะ
A (gold): ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกิ
[2/20] latency=22.10s
Q: ถ้าบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร?
A (model): คำตอบ: หากบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร คือ บุคคลนั้นจะต้องรับโทษปรับไม่เกินหนึ่งหมื่นบาท ตามมาตรา 12 แห่งพระราชบัญญัติทะ
A (gold): การไม่จดทะเบียนพาณิชย์ถือเป็นการฝ่าฝืนหน้าที่ตามกฎหมาย อาจถูกปรับตามอัตราที่กำหนด รวมท

In [21]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
FT_ADAPTER_PATH = "qwen2.5-7b-legal-Ponprapa-v2"

BGE_MODEL_NAME = "BAAI/bge-m3"
TOP_K = 3
MAX_NEW_TOKENS = 256
TEMPERATURE = 0.0


In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


tokenizer = AutoTokenizer.from_pretrained(FT_ADAPTER_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)


model = PeftModel.from_pretrained(base_model, FT_ADAPTER_PATH)
model.eval()

print("Loaded fine-tuned model from:", FT_ADAPTER_PATH)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded fine-tuned model from: qwen2.5-7b-legal-Ponprapa-v2


In [27]:
def build_ft_prompt(question: str) -> str:
    """
    สร้าง prompt สำหรับ FT-only:
    - โครงสร้างคล้าย RAG แต่ไม่มี context
    - มี system บอกบทบาท + คำถามด้านล่าง
    """
    system = (
        "คุณเป็นผู้ช่วยด้านกฎหมายไทยสำหรับประชาชนทั่วไป "
        "ให้ตอบคำถามโดยใช้ความรู้ด้านกฎหมายไทยอย่างระมัดระวัง "
        "ถ้าข้อมูลไม่เพียงพอ ให้บอกอย่างตรงไปตรงมาและอธิบายให้เข้าใจง่าย "
        "ถ้าอ้างมาตรา ให้ระบุชื่อกฎหมายและมาตราให้ชัดเจน\n\n"
    )

    user_part = (
        f"คำถาม:\n{question}\n\n"
        "โปรดตอบเป็นภาษาไทย โดยอธิบายให้เข้าใจง่าย:"
    )

    return system + user_part


In [28]:
import time
import json
import torch

def run_eval_ft(
    data,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
):
    outputs = []
    latencies = []

    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    for i, ex in enumerate(data, start=1):
        q = ex["question"]
        qid = ex.get("id", i)

        # 1) prompt FT-only
        prompt = build_ft_prompt(q)

        # 2) tokenize
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 3) generate + latency
        if device == "cuda":
            torch.cuda.synchronize()
        start = time.time()

        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=(temperature > 0),
                temperature=temperature if temperature > 0 else 1.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        if device == "cuda":
            torch.cuda.synchronize()
        end = time.time()

        latency = end - start
        latencies.append(latency)


        generated = gen_ids[0][inputs["input_ids"].shape[1]:]
        answer_text = tokenizer.decode(generated, skip_special_tokens=True).strip()

        outputs.append({
            "id": qid,
            "question": q,
            "model_answer": answer_text,
            "gold_answer": ex.get("gold_answer", None),
            "latency_sec": latency,
        })

        if i % 5 == 0:
            print(f"Done {i}/{len(data)}")

    if device == "cuda":
        peak_mem_gb = torch.cuda.max_memory_allocated() / (1024 ** 3)
    else:
        peak_mem_gb = None

    summary = {
        "model_name": "Qwen2.5-7B-Instruct + LoRA (Ponprapa v2)",
        "num_samples": len(data),
        "avg_latency_sec": sum(latencies) / len(latencies),
        "peak_vram_gb": peak_mem_gb,
    }

    return outputs, summary


In [29]:
ft_outputs, ft_summary = run_eval_ft(eval_data)

print("==== FT-ONLY SUMMARY ====")
print(ft_summary)

OUTPUT_PATH_FT = "results_ft_model.jsonl"
with open(OUTPUT_PATH_FT, "w", encoding="utf-8") as f:
    for ex in ft_outputs:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("Saved to", OUTPUT_PATH_FT)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Done 5/20
Done 10/20
Done 15/20
Done 20/20
==== FT-ONLY SUMMARY ====
{'model_name': 'Qwen2.5-7B-Instruct + LoRA (Ponprapa v2)', 'num_samples': 20, 'avg_latency_sec': 22.425798344612122, 'peak_vram_gb': 13.831871509552002}
Saved to results_ft_model.jsonl


In [26]:
!pip install rouge_score bert_score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00


In [34]:
from rouge_score import rouge_scorer
from bert_score import score as bertscore

RESULT_PATH = "results_ft_model.jsonl"

preds = []
refs  = []

with open(RESULT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        ex = json.loads(line)
        if ex.get("gold_answer"):
            preds.append(ex["model_answer"])
            refs.append(ex["gold_answer"])

print("Samples with gold:", len(preds))

scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)
rougeL_scores = [scorer.score(r, p)["rougeL"].fmeasure for p, r in zip(preds, refs)]
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

P, R, F1 = bertscore(preds, refs, lang="th")
avg_bert_F1 = float(F1.mean())

print("FT ROUGE-L:", avg_rougeL)
print("FT BERTScore F1:", avg_bert_F1)

Samples with gold: 20
FT ROUGE-L: 0.0
FT BERTScore F1: 0.7187309265136719


In [33]:
with open("results_ft_model.jsonl", "r", encoding="utf-8") as f:
    rows = [json.loads(l) for l in f]

for r in rows[:3]:
    print("Q:", r["question"])
    print("A (model):", r["model_answer"])
    print("A (gold) :", r["gold_answer"])
    print("latency:", r["latency_sec"])
    print("=" * 80)

Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
A (model): ตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1089 ห้างหุ้นส่วนสามัญมีลักษณะที่หุ้นส่วนทุกคนจะต้องรับผิดชอบต่อหนี้สินของห้างหุ้นส่วนนั้น แม้ว่าจะเป็นหนี้สินที่เกิดขึ้นหลังจากตนได้แยกออกจากการเป็นหุ้นส่วนแล้วก็ตาม นอกจากนี้ หุ้นส่วนทุกคนยังต้องรับผิดชอบต่อหนี้สินของห้างหุ้นส่วนนั้น แม้ว่าจะเป็นหนี้สินที่เกิดขึ้นก่อนที่ตนได้เข้าเป็นหุ้นส่วนก็ตาม แต่หากหุ้นส่วนที่รับผิดชอบได้รับอนุญาตจากห้างหุ้นส่วนนั้นให้รับผิดชอบเฉพาะในจำนวนเงินที่ได้รับชำระแล
A (gold) : ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.
latency: 22.76600742340088
Q: ถ้าบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร?
A (model):

In [45]:
!pip install -q peft bitsandbytes datasets faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 108.0 MB/s eta 0:00:00


In [36]:

FT_ADAPTER_PATH = "qwen2.5-7b-legal-Ponprapa-v2"
EVAL_PATH       = "eval_questions.jsonl"


BASE_MODEL_NAME_RAW = "Qwen/Qwen2.5-7B-Instruct"
BGE_MODEL_NAME      = "BAAI/bge-m3"
TOP_K               = 3
MAX_NEW_TOKENS      = 256
TEMPERATURE         = 0.0

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [37]:
import json

eval_data = []
with open(EVAL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        ex = json.loads(line)
        eval_data.append(ex)

print("Loaded eval samples:", len(eval_data))
print(eval_data[0])


Loaded eval samples: 20
{'id': 1, 'question': 'ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?', 'gold_answer': 'ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.'}


In [38]:
from datasets import load_dataset

ds = load_dataset("airesearch/WangchanX-Legal-ThaiCCL-RAG")
train_ds = ds["train"]
print("Columns:", train_ds.column_names)
print("Example[0]:", train_ds[0])


Columns: ['question', 'positive_contexts', 'hard_negative_contexts', 'positive_answer', 'hard_negative_answer']
Example[0]: {'question': 'ธุรกิจสถาบันการเงินและธุรกิจทางการเงินต่างกันอย่างไร', 'positive_contexts': [{'context': 'พระราชบัญญัติธุรกิจสถาบันการเงิน พ.ศ. 2551 มาตรา 4 ในพระราชบัญญัตินี้\n“ธุรกิจสถาบันการเงิน” หมายความว่า ธุรกิจธนาคารพาณิชย์ ธุรกิจเงินทุน และธุรกิจเครดิตฟองซิเอร์ และให้หมายความรวมถึงการประกอบธุรกิจของสถาบันการเงินเฉพาะกิจ\n“ธุรกิจธนาคารพาณิชย์” หมายความว่า การประกอบธุรกิจรับฝากเงินหรือรับเงินจากประชาชนที่ต้องจ่ายคืนเมื่อทวงถาม หรือเมื่อสิ้นระยะเวลาอันกำหนดไว้ และใช้ประโยชน์จากเงินนั้นโดยวิธีหนึ่งวิธีใด เช่น ให้สินเชื่อ ซื้อขายตั๋วแลกเงินหรือตราสารเปลี่ยนมืออื่นใด ซื้อขายเงินปริวรรตต่างประเทศ\n“ธุรกิจเงินทุน” หมายความว่า การประกอบธุรกิจรับฝากเงินหรือรับเงินจากประชาชนที่ต้องจ่ายคืนเมื่อทวงถาม หรือเมื่อสิ้นระยะเวลาอันกำหนดไว้ ซึ่งมิใช่การรับฝากเงินหรือรับเงินไว้ในบัญชีที่จะเบิกถอนโดยใช้เช็ค และใช้ประโยชน์จากเงินนั้นโดยวิธีหนึ่งวิธีใด เช่น ให้สินเชื่อ ซื้อขายตั๋วแ

In [42]:
CONTEXT_COL = "positive_contexts"

corpus_texts = []

for ex in train_ds:
    ctx_list = ex[CONTEXT_COL]
    if not ctx_list:
        continue

    for c in ctx_list:
        text = c.get("context", None)
        if text and isinstance(text, str):
            corpus_texts.append(text)

print("Corpus size:", len(corpus_texts))
print("Sample context:", corpus_texts[0][:300])



Corpus size: 12109
Sample context: พระราชบัญญัติธุรกิจสถาบันการเงิน พ.ศ. 2551 มาตรา 4 ในพระราชบัญญัตินี้
“ธุรกิจสถาบันการเงิน” หมายความว่า ธุรกิจธนาคารพาณิชย์ ธุรกิจเงินทุน และธุรกิจเครดิตฟองซิเอร์ และให้หมายความรวมถึงการประกอบธุรกิจของสถาบันการเงินเฉพาะกิจ
“ธุรกิจธนาคารพาณิชย์” หมายความว่า การประกอบธุรกิจรับฝากเงินหรือรับเงินจากประช


In [46]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

bge = SentenceTransformer(BGE_MODEL_NAME, device=device)

embeddings = bge.encode(
    corpus_texts,
    batch_size=64,
    convert_to_numpy=True,
    show_progress_bar=True,
)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
faiss.normalize_L2(embeddings)
index.add(embeddings)

print("FAISS index size:", index.ntotal)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/190 [00:00<?, ?it/s]

FAISS index size: 12109


In [47]:
def retrieve_bge(query: str, top_k: int = TOP_K):
    q_emb = bge.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        results.append({
            "context": corpus_texts[idx],
            "score": float(score),
        })
    return results

test_q = eval_data[0]["question"]
ret = retrieve_bge(test_q, top_k=1)[0]["context"]
print("Q:", test_q)
print("Retrieved context:", ret[:300])


Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
Retrieved context: ประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1025
อันว่าห้างหุ้นส่วนสามัญนั้น คือห้างหุ้นส่วนประเภทซึ่งผู้เป็นหุ้นส่วนหมดทุกคนต้องรับผิดร่วมกันเพื่อหนี้ทั้งปวงของหุ้นส่วนโดยไม่มีจำกัด


In [48]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# loading tokenizer from LoRA adapter
tokenizer = AutoTokenizer.from_pretrained(FT_ADAPTER_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,   # ถ้า error ค่อยเปลี่ยนเป็น torch.float16
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# base Qwen
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME_RAW,
    quantization_config=bnb_config,
    device_map="auto",
)

# My LoRA
model = PeftModel.from_pretrained(base_model, FT_ADAPTER_PATH)
model.eval()

BASE_MODEL_NAME = "Qwen2.5-7B-Instruct + LoRA (Ponprapa v2)"

print("Loaded FT model:", BASE_MODEL_NAME)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded FT model: Qwen2.5-7B-Instruct + LoRA (Ponprapa v2)


In [49]:
def build_rag_prompt(question: str, retrieved_contexts, max_ctx: int = TOP_K) -> str:
    """
    สร้าง prompt สำหรับ RAG:
    - ใส่ context ที่ retrieve ได้ด้านบน
    - ใส่คำสั่งให้ยึดตาม context เป็นหลัก
    """
    system = (
        "คุณเป็นผู้ช่วยด้านกฎหมายไทยสำหรับประชาชนทั่วไป "
        "ให้ตอบคำถามโดยอ้างอิงจาก 'บริบท' ที่ให้ด้านล่างเท่านั้น "
        "ถ้าในบริบทไม่มีข้อมูลเพียงพอ ให้ตอบอย่างระมัดระวังและบอกว่าข้อมูลอาจไม่ครบถ้วน "
        "ควรอธิบายให้เข้าใจง่าย และถ้าอ้างมาตราให้ยึดจากข้อความในบริบทเท่านั้น\n\n"
    )

    ctx_parts = []
    for i, ctx in enumerate(retrieved_contexts[:max_ctx], start=1):
        ctx_parts.append(f"[เอกสารที่ {i}]\n{ctx['context']}")
    ctx_block = "\n\n".join(ctx_parts)

    user_part = (
        f"บริบทจากเอกสารกฎหมาย:\n{ctx_block}\n\n"
        f"คำถาม:\n{question}\n\n"
        "โปรดตอบเป็นภาษาไทย โดยอ้างอิงจากบริบทด้านบน:"
    )

    return system + user_part


In [50]:
import time
import json

def run_eval_rag_ft(
    data,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    top_k: int = TOP_K,
):
    outputs = []
    latencies = []

    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    for i, ex in enumerate(data, start=1):
        q = ex["question"]
        qid = ex.get("id", i)

        # 1) use context from BGE+FAISS
        retrieved = retrieve_bge(q, top_k=top_k)

        # 2) prompt
        prompt = build_rag_prompt(q, retrieved_contexts=retrieved, max_ctx=top_k)

        # 3) tokenize
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 4) generate + latency
        if device == "cuda":
            torch.cuda.synchronize()
        start = time.time()

        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=(temperature > 0),
                temperature=temperature if temperature > 0 else 1.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        if device == "cuda":
            torch.cuda.synchronize()
        end = time.time()

        latency = end - start
        latencies.append(latency)

        generated = gen_ids[0][inputs["input_ids"].shape[1]:]
        answer_text = tokenizer.decode(generated, skip_special_tokens=True).strip()

        outputs.append({
            "id": qid,
            "question": q,
            "model_answer": answer_text,
            "gold_answer": ex.get("gold_answer", None),
            "latency_sec": latency,
            "retrieved": retrieved,
        })

        if i % 5 == 0:
            print(f"Done {i}/{len(data)}  latency={latency:.2f}s")

    if device == "cuda":
        peak_mem_gb = torch.cuda.max_memory_allocated() / (1024 ** 3)
    else:
        peak_mem_gb = None

    summary = {
        "model_name": BASE_MODEL_NAME,
        "retriever": BGE_MODEL_NAME,
        "num_samples": len(data),
        "avg_latency_sec": sum(latencies) / len(latencies),
        "peak_vram_gb": peak_mem_gb,
        "top_k": top_k,
    }

    return outputs, summary


In [51]:
rag_ft_outputs, rag_ft_summary = run_eval_rag_ft(eval_data)

print("==== RAG + FT SUMMARY ====")
print(rag_ft_summary)

OUTPUT_PATH_RAG_FT = "results_rag_bge_ft.jsonl"
with open(OUTPUT_PATH_RAG_FT, "w", encoding="utf-8") as f:
    for ex in rag_ft_outputs:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("Saved to", OUTPUT_PATH_RAG_FT)


Done 5/20  latency=22.36s
Done 10/20  latency=22.44s
Done 15/20  latency=22.35s
Done 20/20  latency=22.12s
==== RAG + FT SUMMARY ====
{'model_name': 'Qwen2.5-7B-Instruct + LoRA (Ponprapa v2)', 'retriever': 'BAAI/bge-m3', 'num_samples': 20, 'avg_latency_sec': 22.339265835285186, 'peak_vram_gb': 16.230682373046875, 'top_k': 3}
Saved to results_rag_bge_ft.jsonl


In [52]:
with open("results_rag_bge_ft.jsonl", "r", encoding="utf-8") as f:
    rows = [json.loads(l) for l in f]

for r in rows[:3]:
    print("Q:", r["question"])
    print("A (model):", r["model_answer"])
    print("A (gold) :", r["gold_answer"])
    print("latency:", r["latency_sec"])
    print("=" * 80)

Q: ห้างหุ้นส่วนสามัญต่างจากห้างหุ้นส่วนจำกัดในแง่ความรับผิดของหุ้นส่วนอย่างไร ตามประมวลกฎหมายแพ่งและพาณิชย์?
A (model): ห้างหุ้นส่วนสามัญและห้างหุ้นส่วนจำกัดมีความแตกต่างในเรื่องความรับผิดของหุ้นส่วน โดยห้างหุ้นส่วนสามัญนั้น ผู้เป็นหุ้นส่วนหมดทุกคนต้องรับผิดร่วมกันเพื่อหนี้ทั้งปวงของหุ้นส่วนโดยไม่มีจำกัด (ตามมาตรา 1025) ในขณะที่ห้างหุ้นส่วนจำกัดที่ยังไม่ได้จดทะเบียน จะถือว่าเป็นห้างหุ้นส่วนสามัญซึ่งผู้เป็นหุ้นส่วนทั้งหมดย่อมต้องรับผิดร่วมกันในบรรดาหนี้ของห้างหุ้นส่วนโดยไม่มีจำกัดจำนวน จนกว่าจะได้จดทะเบียน (ตามมาตรา 1079) ดังนั้น ความรับผิดของหุ
A (gold) : ห้างหุ้นส่วนสามัญ หุ้นส่วนทุกคนต้องรับผิดไม่จำกัดและร่วมกันด้วยทรัพย์สินส่วนตัวต่อหนี้ของห้างหุ้นส่วน ส่วนห้างหุ้นส่วนจำกัดมีทั้งหุ้นส่วนผู้จัดการที่รับผิดไม่จำกัด และหุ้นส่วนจำกัดที่รับผิดเพียงไม่เกินจำนวนเงินที่ตนลงทุน จึงเป็นโครงสร้างที่จำกัดความเสี่ยงของผู้ร่วมลงทุนบางส่วน.
latency: 22.147130012512207
Q: ถ้าบุคคลตั้งห้างหุ้นส่วนหรือบริษัทแต่ไม่ไปจดทะเบียนพาณิชย์ตามพระราชบัญญัติทะเบียนพาณิชย์ จะมีผลทางกฎหมายอย่างไร?
A (model): ตามพ

# **Evaluation Summary: BASE vs RAG vs Fine-Tuning vs RAG+FT**

## **1. Performance Overview (20-item Thai Legal QA Benchmark)**

| Model Variant | RAG | Fine-tuned | ROUGE-L | BERTScore F1 | Avg Latency (s) | Peak VRAM (GB) |
|--------------|-----|------------|----------|---------------|------------------|------------------|
| **BASE (Qwen2.5-7B-Instruct)** | x | x | ~0.006 | ~0.719 | ~30.8 | ~14.2 |
| **RAG (BGE-M3 + WangchanX corpus)** | ✓ | x | ~0.0* | ~0.710 | ~31.4 | ~16.7 |
| **Fine-Tuned Only (LoRA)** | x | ✓ | ~0.0* | ~0.719 | ~22.7 | ~14.0 |
| **RAG + Fine-Tuned (Final Model)** | ✓ | ✓ | ~0.0* | ~0.718 | **~22.3 (fastest)** | ~16.2 |

\*ROUGE-L = 0.0 is due to Thai tokenization limitations in the default ROUGE implementation.

---

## **2. Qualitative Behavior**

### **BASE Model**
- Frequently hallucinates legal sections  
- Produces generic or off-topic reasoning  
- High variability in answer structure

### **RAG-Only**
- Strong factual grounding from retrieved law text  
- Still overly verbose  
- Occasionally misaligns with Thai legal writing conventions

### **Fine-Tuned Only**
- Answers follow Thai legal style more closely  
- Less hallucination  
- Limited by scope of training data since no retrieval

### **RAG + Fine-Tuned (Best Overall)**
- Combines precision of retrieval with legal-style reasoning  
- Most consistent factual accuracy  
- Minimal hallucination  
- Fastest and most stable inference behavior

---

## **3. Notes on ROUGE-L Scores**

The default `rouge_score` implementation is English-centric and uses word-level tokenization.  
For Thai, the entire sentence becomes a single token → ROUGE-L ≈ 0.  
This does *not* reflect the model’s real performance.

**BERTScore F1** (~0.71 across models) is the meaningful metric for Thai.

---

## **4. Key Insights**

- **RAG → better factual correctness**  
- **Fine-tuning → better legal reasoning style + less hallucination**  
- **RAG + FT → best combined performance**

Summary:

> **RAG = grounding**  
> **FT = legal style + stability**  
> **RAG+FT = production-quality Thai legal QA**

---




### Human Evaluation Summary (Legal QA)

Below is the human-scored evaluation for 4 model configurations:

- **1 = RAG + FT**
- **2 = FT-only**
- **3 = RAG-only**
- **4 = Base model**

#### Scoring Rubric (1–5)
- **A — Legal Correctness:** Accuracy of law, correct sections, correct legal principles  
- **B — Reasoning Structure:** Separation of facts/law, coherent ordering  
- **C — Completeness:** Captures all legally relevant points  
- **D — Clarity:** Readability, logical flow  

---

### Evaluation Table

| Model | A. Legal Correctness | B. Reasoning Structure | C. Completeness | D. Clarity | Notes |
|-------|-----------------------|------------------------|------------------|------------|--------|
| **1) RAG + FT** | **4** | 3 | 3 | 3 | Mostly correct; RAG helps citations; some truncation + mild repetition |
| **2) FT-only** | **2** | 3 | 2 | 3 | Structure okay but several wrong legal sections / wrong thresholds |
| **3) RAG-only** | **5** | 4 | 4 | 4 | Most accurate overall; strong factual grounding from retrieved law |
| **4) Base model** | **2** | 3 | 3 | 3 | General reasoning intact but factual mistakes (e.g., legal entity mixing, wrong %) |

---

### Interpretation
- **RAG-only (3)** is the most reliable in legal correctness—retrieval anchors the answer.  
- **RAG + FT (1)** improves but still suffers from some FT noise and hallucinated merges.  
- **FT-only (2)** shows instability: format good but factual mistakes appear.  
- **Base (4)** shows typical LLM hallucinations on Thai law—mixing concepts, wrong thresholds.

---



#Exaple of answer
Model Answer
##Question:

How many shares must shareholders of a public limited company hold in order to request the board of directors to call an Extraordinary General Meeting (EGM)?

##Gold Answer:

As a rule, shareholders must jointly hold not less than 10% of the total sold shares to request the board of directors to call an extraordinary general meeting. The request must specify the reasons and the proposed agenda.

###1)Model Answer (FT+rag):

Shareholders of a public limited company may request that the board of directors call a shareholders’ meeting when one or more shareholders jointly hold not less than 10% of the total number of sold shares. They may submit a written request asking the board to convene an extraordinary general meeting at any time, pursuant to Section 100 of the Public Limited Companies Act B.E. 2535.
The written request must clearly specify the matters and reasons for calling the meeting.

###2)Model Answer (FT):

According to Section 148 of the Public Limited Companies Act B.E. 2535, shareholders may request the board of directors to call an extraordinary general meeting when they hold not less than 5% of the total issued shares.
The shareholders must also notify the board at least fifteen days prior to the date of the extraordinary meeting. This means that shareholders must participate by submitting a request and providing advance notice as stipulated in Section 148.

###3)Model Answer (RAG):
One or more shareholders who jointly hold not less than 10% of the total number of sold shares may request the board of directors to call an extraordinary general meeting. The written request must specify the matters and reasons for calling the meeting.
Under Section 100 of the Public Limited Companies Act B.E. 2535, if the board does not convene the meeting within forty-five days from receiving the request, the shareholders who signed the request—or other shareholders holding the required percentage—may convene the meeting themselves within forty-five days from the end of that period.

###4)Model Answer (base model):

Under Thai law, shareholders of a public limited company may request the board of directors to call an extraordinary general meeting under Section 128 of the Business Corporations Act B.E. 2558, which states that shareholders holding not less than 3% of the registered capital may request that the board convene an extraordinary meeting within sixty days of the request.
However, this requirement may change depending on amendments to relevant laws, so it is advisable to verify the latest information with the Securities and Exchange Commission (SEC) or consult a lawyer specializing in corporate law.